# Problem 10

Task: Write scripts to backup and restore the database.

- Create a backup of the current database.
- Write a script to restore the database from the backup file.

### Database Creation

In [45]:
import sqlite3

con = sqlite3.connect("heroes_villains.db", timeout=10)
cur = con.cursor()

req_check_tables = cur.execute("SELECT name FROM sqlite_master")
res_check_tables = req_check_tables.fetchall()

for table in res_check_tables:
    if table[0] != 'sqlite_sequence':
        cur.execute(f"DROP TABLE IF EXISTS {table[0]}")

con.commit()    

cur.execute("""
        CREATE TABLE heroes (
            hero_id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            release_date INTEGER NOT NULL,
            hero_name TEXT NOT NULL,
            powers TEXT NOT NULL
        )
    """)
con.commit()

cur.execute("""
        CREATE TABLE villains(
            villain_id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            release_date INTEGER NOT NULL,
            villain_name TEXT NOT NULL,
            powers TEXT NOT NULL
        )
    """)
con.commit()

cur.execute("""
        CREATE TABLE arch_nemesis(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            hero_id INTEGER NOT NULL,
            villain_id INTEGER NOT NULL,
            FOREIGN KEY(hero_id) REFERENCES heroes(hero_id)
            FOREIGN KEY(villain_id) REFERENCES villains(villain_id)
        )
""")
con.commit()

req_fetch_all = cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
req_fetch_all.fetchall()

[('sqlite_sequence',), ('heroes',), ('villains',), ('arch_nemesis',)]

### Problem 1 Solution - create a table for superheroes 

In [46]:
heroes = [
    {'first_name':'Bruce', 'last_name':'Vayne', 'release_date':1939, 'hero_name':'Batman','powers':'Intelligent, Rich, Power Armor'},
    {'first_name':'Barry', 'last_name': 'Allan', 'release_date':1940, 'hero_name':'The Flash','powers':'Superspeed, Lightning Manipulation, Speed Force'},
    {'first_name':'Tony', 'last_name':'Stark','release_date':1963, 'hero_name':'Ironman','powers':'Power Armor, Intelligent, Rich'},
    {'first_name':'Peter', 'last_name':'Parker','release_date':1962,'hero_name':'Spiderman','powers':'Super Sence, Spiderweb, athletic'},
]

# check if table has entries

req_check_heroes_table = cur.execute("SELECT COUNT(*) FROM heroes")
res_check_heroes_table = req_check_heroes_table.fetchone()[0]

if res_check_heroes_table == 0:
    for hero in heroes:
        cur.execute("""
                    INSERT INTO heroes (first_name, last_name, release_date, hero_name, powers)
                    VALUES (?, ?, ?, ?, ?)
                    """, (hero['first_name'], hero['last_name'], hero['release_date'], hero['hero_name'], hero['powers']))
        con.commit()  
        
for row in cur.execute("SELECT * FROM heroes"):
    print(row)                   

(1, 'Bruce', 'Vayne', 1939, 'Batman', 'Intelligent, Rich, Power Armor')
(2, 'Barry', 'Allan', 1940, 'The Flash', 'Superspeed, Lightning Manipulation, Speed Force')
(3, 'Tony', 'Stark', 1963, 'Ironman', 'Power Armor, Intelligent, Rich')
(4, 'Peter', 'Parker', 1962, 'Spiderman', 'Super Sence, Spiderweb, athletic')


### Prolem 2 Solution - Create a new table for supervillains

In [47]:
villains = [
    {'first_name':'Arthur','last_name':'Fleck','release_date':2019,'villain_name':'Joker','powers':'Master Manipulator, Chemical Engineer, Master Planner'},
    {'first_name':'Eobard','last_name':'Thawne','release_date':1963,'villain_name':'Reverse Flash','powers':'Negative Speed Force, Superspeed, Lightning Manipulation'},
    {'first_name':'Gene','last_name':'Kahn','release_date':1964,'villain_name':'Mandarin','powers':'Genius-level Intellect, Master of Martial Arts, Expert Strategist and Tactician'},
    {'first_name':'Norman','last_name':'Osborn','release_date':1964,'villain_name':'Green Goblin','powers':'Superhuman Strength, Superhuman Speed, Superhuman Refelexes'},
]

req = cur.execute("SELECT COUNT(*) FROM villains")
res = cur.fetchone()[0]

if res == 0:
    for villain in villains:
        cur.execute("""
                    INSERT INTO villains (first_name, last_name, release_date, villain_name, powers)
                    VALUES (?, ?, ?, ?, ?)
                    """, (villain['first_name'], villain['last_name'], villain['release_date'], villain['villain_name'], villain['powers']))
        con.commit()

for row in cur.execute("SELECT * FROM villains"):
    print(row)        

(1, 'Arthur', 'Fleck', 2019, 'Joker', 'Master Manipulator, Chemical Engineer, Master Planner')
(2, 'Eobard', 'Thawne', 1963, 'Reverse Flash', 'Negative Speed Force, Superspeed, Lightning Manipulation')
(3, 'Gene', 'Kahn', 1964, 'Mandarin', 'Genius-level Intellect, Master of Martial Arts, Expert Strategist and Tactician')
(4, 'Norman', 'Osborn', 1964, 'Green Goblin', 'Superhuman Strength, Superhuman Speed, Superhuman Refelexes')


### Problem 3 Solution - create a table to establish reletianship between heroes and villains

In [48]:
# get hero id's
req_heroes_ids = cur.execute("SELECT hero_id FROM heroes")
res_heroes_ids = req_heroes_ids.fetchall()

# get villains id's
req_villains = cur.execute("SELECT villain_id FROM villains")
res_villains_ids = req_villains.fetchall()

for hero_id, villain_id in zip(res_heroes_ids, res_villains_ids):
    cur.execute("""
                INSERT INTO arch_nemesis (hero_id, villain_id)
                VALUES (?, ?)
                """, (hero_id[0], villain_id[0]))
    con.commit()

for row in cur.execute("SELECT * FROM arch_nemesis"):
    print(row)    

(1, 1, 1)
(2, 2, 2)
(3, 3, 3)
(4, 4, 4)


### Problem 4 Solution - Update the powers of a chosen hero or villain

In [49]:
import random

cur.execute(f"UPDATE heroes SET powers='NEW, RANDOM, POWERS, !' WHERE hero_id={random.randint(1,4)}")

con.commit()

for row in cur.execute("SELECT * FROM heroes"):
    print(row)

(1, 'Bruce', 'Vayne', 1939, 'Batman', 'Intelligent, Rich, Power Armor')
(2, 'Barry', 'Allan', 1940, 'The Flash', 'Superspeed, Lightning Manipulation, Speed Force')
(3, 'Tony', 'Stark', 1963, 'Ironman', 'NEW, RANDOM, POWERS, !')
(4, 'Peter', 'Parker', 1962, 'Spiderman', 'Super Sence, Spiderweb, athletic')


### Problem 5 Solution - Delete a record

In [50]:
import random

cur.execute(f"DELETE FROM heroes WHERE hero_id={random.randint(1,4)}")

con.commit()

for row in cur.execute("SELECT * FROM heroes"):
    print(row)

(2, 'Barry', 'Allan', 1940, 'The Flash', 'Superspeed, Lightning Manipulation, Speed Force')
(3, 'Tony', 'Stark', 1963, 'Ironman', 'NEW, RANDOM, POWERS, !')
(4, 'Peter', 'Parker', 1962, 'Spiderman', 'Super Sence, Spiderweb, athletic')


### Problem 6 Solution - query with parameters (before certain release year)

In [51]:
import random

print("FROM HEROES TABLE:")
for row in cur.execute("SELECT * FROM heroes WHERE release_date > 1959"):
    print(row)

print("FROM VILLAINS TABLE:")
for row in cur.execute("SELECT * FROM villains WHERE release_date < 1959"):
    print(row)

FROM HEROES TABLE:
(3, 'Tony', 'Stark', 1963, 'Ironman', 'NEW, RANDOM, POWERS, !')
(4, 'Peter', 'Parker', 1962, 'Spiderman', 'Super Sence, Spiderweb, athletic')
FROM VILLAINS TABLE:


### Problem 7 Solution - Join querry

In [52]:
# Join query to find the arch-nemesis of each hero using INNER JOIN and USING
cur.execute("""
    SELECT
        heroes.hero_name AS hero,
        villains.villain_name AS villain
    FROM
        arch_nemesis
    INNER JOIN
        heroes USING (hero_id)
    INNER JOIN    
        villains USING (villain_id);    
""")

# Print the results
for row in cur.fetchall():
    print(f"hero: {row[0]}, arch nemmesis: {row[1]}")


hero: The Flash, arch nemmesis: Reverse Flash
hero: Ironman, arch nemmesis: Mandarin
hero: Spiderman, arch nemmesis: Green Goblin


### Problem 8 Solution - Create and use indexes

In [53]:
import time

cur.execute("DROP TABLE IF EXISTS idx_release_date")

cur.execute("""
            CREATE INDEX 
                idx_release_date
            ON
                heroes (release_date)
            
            """)
con.commit()

# Measure time using the index
start = time.process_time()
for row in cur.execute("SELECT * FROM heroes ORDER BY release_date;"):
    print(row)
end = time.process_time()
print(f"Time using index: {end - start}")

# Measure time without using the index
start = time.process_time()
for row in cur.execute("SELECT release_date FROM heroes;"):
    print(row)
end = time.process_time()
print(f"Time using heroes table: {end - start}")

(2, 'Barry', 'Allan', 1940, 'The Flash', 'Superspeed, Lightning Manipulation, Speed Force')
(4, 'Peter', 'Parker', 1962, 'Spiderman', 'Super Sence, Spiderweb, athletic')
(3, 'Tony', 'Stark', 1963, 'Ironman', 'NEW, RANDOM, POWERS, !')
Time using index: 0.0
(1940,)
(1962,)
(1963,)
Time using heroes table: 0.0


### Problem 9 Solution - aggregate functions

In [54]:
req_average_release_year = cur.execute("SELECT avg(release_date) FROM heroes")
res_average_release_year = req_average_release_year.fetchone()[0]
print(f"Average release year for heroes: {res_average_release_year}")

power_to_seach = 'Intelligent'
req_specific_power = cur.execute("SELECT COUNT(*) FROM heroes WHERE powers LIKE ?", ('%' + power_to_seach + '%',))
res_specific_power = cur.fetchone()[0]
print(f"Number of heroes with {power_to_seach}: {res_specific_power}")

power_to_search = 'Super'
req_specific_power = cur.execute("SELECT COUNT(*) FROM villains WHERE powers LIKE ?", ('%' + power_to_search + '%',))
res_specific_power = req_specific_power.fetchone()[0]
print(f"Number of heroes with {power_to_search}: {res_specific_power}")

Average release year for heroes: 1955.0
Number of heroes with Intelligent: 0
Number of heroes with Super: 2


### Problem 10 Solution - Back and restore

In [55]:
import io

with io.open('heroes_villains_backup.sql', 'w') as p:
    for line in con.iterdump():
        p.write('{}\n'.format(line))
print('Backup performed successfully!')
print('Data saved as heroes_villains_backup.sql')

Backup performed successfully!
Data saved as heroes_villains_backup.sql
